In [6]:
!pip install selenium
!pip install parsel
!pip install pathlib
!pip install numpy
!pip install pandas
!pip install bs4
!pip install requests

  Using cached parsel-1.6.0-py2.py3-none-any.whl (13 kB)
  Using cached w3lib-1.22.0-py2.py3-none-any.whl (20 kB)
  Using cached cssselect-1.1.0-py2.py3-none-any.whl (16 kB)
  Created wheel for pathlib: filename=pathlib-1.0.1-py3-none-any.whl size=14347 sha256=888e40e51980e91f1884f550edf02718635ffa17aac49aa10ec62f1f5d18f147
  Stored in directory: c:\users\genext\appdata\local\pip\cache\wheels\59\02\2f\ff4a3e16a518feb111ae1405908094483ef56fec0dfa39e571
Successfully built pathlib
  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=c20801c11eaa61de92a13156f8d67b60cd7ae1de30d887901a6e8f734cf02845
  Stored in directory: c:\users\genext\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [7]:
# Given the profile urls scrape skills and other info

from selenium import webdriver
from parsel import Selector
import time
import pathlib

import requests,json,re,datetime
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd



In [8]:
chromedriver = str(pathlib.Path().resolve())+'/chromedriver.exe' #Put your chromedriver.exe into current directory as MAC and windows have different system path 😉

_DRIVER_CHROME = webdriver.Chrome(chromedriver)



_DRIVER_CHROME.get('https://www.linkedin.com/uas/login')

elementID = _DRIVER_CHROME.find_element_by_id('username')
elementID.send_keys('maatsisipbl@gmail.com')

elementID = _DRIVER_CHROME.find_element_by_id('password')
elementID.send_keys('project@123')

elementID.submit()
time.sleep(30) 


In [9]:
def get_json1(url):
	_DRIVER_CHROME.get(url)
	html = _DRIVER_CHROME.page_source
	soup = BeautifulSoup(html , "html.parser")
	for i in soup.findAll('code'):
		if(re.match(r'^\n  {"request":"/voyager/api/identity/dash/profiles.*.FullProfileWithEntities.*',i.text)):
			idd=json.loads(i.text)['body']
	data=soup.find_all(attrs={'id':idd})[0]
	for i in data:
		s=json.loads(i)
	return s['included']
	# print(html)

In [17]:
def get_mm(dd):
    ff={}
    current_month=int(datetime.datetime.today().month)  
    sm=dd['Start Month']
    em=dd['End Month']
    #print(sm,em)

    if sm!='No DateRange Start Month Key' or em!='No DateRange End Month Key':
        if sm!='No Start Month':
            if em!='No End Month' and em!='No end month':
                if em!='Present':
                    if(em<sm):
                        ff['m']=em-sm+1
                    else:
                        ff['m']= np.abs(em-sm)+1
                else:
                    ff['m']=current_month-sm+1
            else:
                ff['m']=0
        else:
                ff['m']= 0
    else:
        ff['m']= 0
    return ff


In [10]:
def get_educ(s1):
    ls=[]
    for i in range(len(s1)):
        d={}
        if(re.match('com.linkedin.voyager.dash.identity.profile.Education',s1[i]['$type'])):
            kk=s1[i]
            #print(kk.keys())
            if 'degreeName' in kk.keys() and 'schoolName' in kk.keys() and 'fieldOfStudy' in kk.keys() and 'dateRange' in kk.keys():
                if(kk['degreeName'] is not None):
                    d['DegreeName']=kk['degreeName']
                else:
                    d['DegreeName']='Not Mentioned'

                if(kk['schoolName'] is not None):
                    d['SchoolName']=kk['schoolName']
                else:
                    d['SchoolName']='Not Mentioned School'

                if(kk['fieldOfStudy'] is not None):
                    d['FieldOfStudy']=kk['fieldOfStudy']
                else:
                    d['FieldOfStudy']='Not Mentioned FS'
                    
                if kk['dateRange'] is not None:
                    if 'start' in kk['dateRange'].keys() and kk['dateRange']['start'] is not None:
                        if 'year' in kk['dateRange']['start'].keys() and kk['dateRange']['start']['year'] is not None:
                            d['Start Date']=kk['dateRange']['start']['year']
                            #dd.append(kk['dateRange']['start']['year'])
                        else:
                            d['Start Date']='No Start Date'
                    else:
                        d['Start Date']='No Start Key'
                        
                    if 'end' in kk['dateRange'].keys() and kk['dateRange']['end'] is not None:
                        if 'year' in kk['dateRange']['end'].keys() and kk['dateRange']['end']['year'] is not None:
                            d['End Date']=kk['dateRange']['end']['year']
                        else:
                            d['End Date']='No end year'
                    else:
                    #l=kk['dateRange']['end']['year']='No end year'
                        d['End Date']='No end key'


            #k=sn+"-"+dn+"-"+fs
            #ls.append(i['schoolName'])
            #ls.append(i['degreeName'])
            #ls.append(i['fieldOfStudy'])
            ls.append(d)
    return ls

In [18]:
def get_skills(s1):
    skills=[]
    for i in range(len(s1)):
        if(re.match('com.linkedin.voyager.dash.identity.profile.Skill',s1[i]['$type'])):
            skills.append(s1[i]['name'])
    return skills

In [19]:
def get_certificates(s1):
    certificates=[]
    for i in range(len(s1)):
        ddd={}
        if(re.match('com.linkedin.voyager.dash.identity.profile.Certification',s1[i]['$type'])):
            kk=s1[i]
            if 'authority' in kk.keys() and 'name' in kk.keys() and kk['name'] is not None:
                if kk['authority'] is not None:
                    ddd['Name']=kk['name']
                    ddd['Authority']=kk['authority']
                    #kk=s1[i]['name']+"-"+s1[i]['authority']
                    certificates.append(ddd)
                else:
                    ddd['Name']=kk['name']
                    ddd['Authority']='No Authority'
                    #kk=s1[i]['name']+"-"+s1[i]['authority']
                    certificates.append(ddd)
                    #kk=s1[i]['name']+"-"+'No Auth'
                    #certificates.append(kk)
    return certificates

In [20]:
def get_volexp(s1):
    volexp=[]
    for i in range(len(s1)):
        if re.match('com.linkedin.voyager.dash.identity.profile.VolunteerExperience',s1[i]['$type']):
            d={}
            kk=s1[i]

            if 'role' in kk.keys() and 'companyName' in kk.keys():
                if kk['role'] is not None:
                    d['Role']=kk['role']
                else:
                    d['Role']='No Role'

                if kk['companyName'] is not None:
                    d['Company Name']=kk['companyName']

                else:
                    d['Company Name']='No Company Name'


            if 'description' in kk and kk['description'] is not None:
                d['Description']=kk['description']
            else:
                d['Description']='No Description'


            #print(tt,loc,dd,cc)
            #print('------------------------------------')
            #Job_details=tt+"-"+cc+"-"+loc+"-"+dd
            #print(Job_details)
            volexp.append(d)
    return volexp

In [21]:
def get_projects(s1):
    projects=[]
    for i in range(len(s1)):
        d={}
        if(re.match('com.linkedin.voyager.dash.identity.profile.Project',s1[i]['$type'])):
            if 'title' in s1[i] and 'description' in s1[i] and s1[i]['title'] is not None:
                if (s1[i]['description'] is not None):
                    d['Title']=s1[i]['title']
                    d['Description']=s1[i]['description']
                    #vv=s1[i]['title']+'-'+s1[i]['description']
                    projects.append(d)
                else:
                    d['Title']=s1[i]['title']
                    d['Description']='No Description'
                    #vv=s1[i]['title']+'-'+s1[i]['description']
                    projects.append(d)
                    #vv=s1[i]['title']+'-'+'No Descc'
                    #projects.append(vv)
    return projects

#projects          #com.linkedin.voyager.dash.identity.profile.Honor

In [22]:
def get_patents(s1):
    patents=[]
    for i in range(len(s1)):
        d={}
        if(re.match('com.linkedin.voyager.dash.identity.profile.Patent',s1[i]['$type'])):
            if 'title' in s1[i].keys() and 'description' in s1[i].keys() and s1[i]['title'] is not None:
                if (s1[i]['description'] is not None):
                    d['Title']=s1[i]['title']
                    d['Description']=s1[i]['title']
                    #vv=s1[i]['title']+'-'+s1[i]['description']
                    patents.append(d)
                else:
                    d['Title']=s1[i]['title']
                    d['Description']='No Description'
                    #vv=s1[i]['title']+'-'+s1[i]['description']
                    patents.append(d)
                    #vv=s1[i]['title']+'-'+'No Descc'
                    #patents.append(vv)
    return patents
            #com.linkedin.voyager.dash.identity.profile.Honor

In [23]:
def get_honors(s1):
    honors=[]
    for i in range(len(s1)):
        d={}
        if(re.match('com.linkedin.voyager.dash.identity.profile.Honor',s1[i]['$type'])):
            kk=s1[i]
            if 'title' in kk.keys() and 'description' in kk.keys() and kk['title'] is not None:
                if (kk['description'] is not None):
                    d['Title']=kk['title']
                    d['Description']=kk['description']
                    #vv=s1[i]['title']+'-'+s1[i]['description']
                    honors.append(d)
                else:
                    d['Title']=kk['title']
                    d['Description']='No Description'
                    #vv=s1[i]['title']+'-'+s1[i]['description']
                    honors.append(d)
                    #vv=s1[i]['title']+'-'+'No Descc'
                    #honors.append(vv)
    return honors
 #com.linkedin.voyager.dash.identity.profile.Honor

In [24]:
def get_publication(s1):
    publications=[]
    for i in range(len(s1)):
        d={}
        if(re.match('com.linkedin.voyager.dash.identity.profile.Publication',s1[i]['$type'])):
            if 'name' in s1[i].keys() and 'publisher' in s1[i].keys() and s1[i]['name'] is not None:
                if (s1[i]['publisher'] is not None):
                    d['Name']=s1[i]['name']
                    d['Publisher']=s1[i]['publisher']
                    #vv=s1[i]['name']+'-'+s1[i]['publisher']
                    publications.append(d)
                else:
                    d['Name']=s1[i]['name']
                    d['Publisher']='No Publisher Name'
                    #vv=s1[i]['name']+'-'+s1[i]['publisher']
                    publications.append(d)
                    #vv=s1[i]['name']+'-'+'No Descc'
                    #publications.append(vv)
    return publications

In [25]:
def get_exp(s1):
    exps=[]
    current_year=int(datetime.datetime.today().year)
    current_month=int(datetime.datetime.today().month)
    for i in range(len(s1)):
        d={}
        if re.match('com.linkedin.voyager.dash.identity.profile.Position$',s1[i]['$type']):
            kk=s1[i]
            if 'title' in kk.keys() and 'companyName' in kk.keys() and kk['title'] is not None and 'dateRange' in kk.keys():
                if kk['companyName'] is not None:
                    d['Title']=kk['title']
                    d['Company Name']=kk['companyName']
                else:
                    d['Title']=kk['title']
                    d['Company Name']='No Company Name'
                
                if kk['dateRange'] is not None:
                    if 'start' in kk['dateRange'].keys() and kk['dateRange']['start'] is not None:
                        if 'year' in kk['dateRange']['start'].keys() and kk['dateRange']['start']['year'] is not None:
                            d['Start Year']=kk['dateRange']['start']['year']
                            #dd.append(kk['dateRange']['start']['year'])
                        else:
                            d['Start Year']='No Start Year'
                            
                        if 'month' in kk['dateRange']['start'].keys() and kk['dateRange']['start']['month'] is not None:
                            d['Start Month']=kk['dateRange']['start']['month']
                            #dd.append(kk['dateRange']['start']['year'])
                        else:
                            d['Start Month']='No Start Month'
                             
                    else:
                        d['Start Year']='No Start Year Key'
                        d['Start Month']='No Start Month Key'
                        
                    if 'end' in kk['dateRange'].keys() and kk['dateRange']['end'] is not None:
                        if 'year' in kk['dateRange']['end'].keys() and kk['dateRange']['end']['year'] is not None:
                            d['End Year']=kk['dateRange']['end']['year']
                        else:
                            d['End Year']='No end year'
                            
                        
                        if 'month' in kk['dateRange']['end'].keys() and kk['dateRange']['end']['month'] is not None:
                            d['End Month']=kk['dateRange']['end']['month']
                        else:
                            d['End Month']='No end month'
                        
                    else:
                    #l=kk['dateRange']['end']['year']='No end year'
                        d['End Year']='Present'
                        d['End Month']='Present'
                else:
                    d['Start Year']='No DateRange Value'
                    d['End Year']='No DateRange Value'
                
            else:
                d['Title']='No title key'
                d['Company Name']='No companyName key'
                d['Start Year']='No DateRange Start Key'
                d['End Year']='No DateRange End Key'
                d['Start Month']='No DateRange Start Month Key'
                d['End Month']='No DateRange End Month Key'
            #print(d)
            if d['Title']=='No title key':
                   continue
            
            #if d['Title']!='No Title Key':            
            if d['Start Year'] is not None and d['Start Year']!='No Start Year' and d['Start Year']!='No DateRange Value' and d['Start Year']!='No DateRange Start Key':
                if d['End Year']!='No end year':
                    d['Exp (in months)']=12*int(current_year-d['Start Year'])
                if d['End Year']!='Present':
                    d['Exp (in months)']=12*int(d['End Year']-d['Start Year'])
                else:
                    d['Exp (in months)']=12*int(current_year-d['Start Year'])
                #print(d)
                jjj=get_mm(d)
                d['Total Exp']=jjj['m']+d['Exp (in months)']
            
            if 'geoLocationName' in kk.keys() and kk['geoLocationName'] is not None:
                d['Location']=kk['geoLocationName']
            else:
                d['Location']='No Location'

            if 'description' in kk.keys() and kk['description'] is not None:
                d['Description']=kk['description']
            else:
                d['Description']='No Description'
            
            
            #print(tt,loc,dd,cc)
            #print('------------------------------------')
            #Job_details=tt+"-"+cc+"-"+loc+"-"+dd
            #print(Job_details)
            #if d['Title']
            exps.append(d)
    return exps

In [11]:
# def get_about(s1):
#     for i in range(len(s1)):
#         if(re.match('com.linkedin.voyager.dash.identity.profile.Profile$',s1[i]['$type'])):
#             if 'summary' in s1[i].keys() and s1[i]['summary'] is not None:
#                 summary=s1[i]['summary']
#                 return summary
#             else:
#                 summary='None'
#                 return summary

In [31]:
def brain(urls):
    main_df=pd.DataFrame()
    for url in urls:
        print(url)
        df=pd.DataFrame()
        s=get_json1(url)
        df['Profile Url']=[url]
        df['Education']=[get_educ(s)]
        df['Experience']=[get_exp(s)]
        df['Skills']=[get_skills(s)]
        df['Certificates']=[get_certificates(s)]
        df['Vol Exp']=[get_volexp(s)]
        df['Projects']=[get_projects(s)]
        df['Patents']=[get_patents(s)]
        df['Honors']=[get_honors(s)]
        df['Publications']=[get_publication(s)]
        main_df=pd.concat([main_df,df],axis=0,ignore_index=True)
    return main_df
# 		df['About']=[get_about(s)]
		# print(s)
		#op=open_to_work(s)
    # if op is not None and op!='dd':
		


    #     
   
    #     
    #         #df=df.from_dict(d)

In [54]:
urls = ['https://www.linkedin.com/in/atharva-parikh-ap07/']
# linkedin_scrape(urls)
# get_json1('https://www.linkedin.com/in/atharva-parikh-ap07/')
pkk=brain(urls)

https://www.linkedin.com/in/atharva-parikh-ap07/


In [55]:
pkk

,Profile Url,Education,Experience,Skills,Certificates,Vol Exp,Projects,Patents,Honors,Publications
0,https://www.linkedin.com/in/atharva-parikh-ap07/,"[{'DegreeName': 'TY btech', 'SchoolName': 'Vis...","[{'Title': 'Jt. Head Treasurer', 'Company Name...","[MySQL, Python (Programming Language), HTML, M...","[{'Name': 'Reading Financial Statements', 'Aut...",[],"[{'Title': 'Low code No code applications', 'D...",[],[],[]


In [56]:
pkk['Education'][0]

[{'DegreeName': 'TY btech',
  'SchoolName': 'Vishwakarma Institute of Information Technology',
  'FieldOfStudy': 'Information Technology',
  'Start Date': 2019,
  'End Date': 2023}]

In [65]:
pkk['Experience'][0]

[{'Title': 'Jt. Head Treasurer',
  'Company Name': 'Information Technocrats Students Forum,VIIT',
  'Start Year': 2020,
  'Start Month': 8,
  'End Year': 2021,
  'End Month': 8,
  'Exp (in months)': 12,
  'Total Exp': 13,
  'Location': 'No Location',
  'Description': 'No Description'},
 {'Title': 'Sponsorship Team Member',
  'Company Name': 'IoT Forum VIIT',
  'Start Year': 2020,
  'Start Month': 8,
  'End Year': 2021,
  'End Month': 1,
  'Exp (in months)': 12,
  'Total Exp': 6,
  'Location': 'Pune, Maharashtra, India',
  'Description': 'No Description'},
 {'Title': 'Core Committee Member',
  'Company Name': 'Machine Learning Forum VIIT',
  'Start Year': 2020,
  'Start Month': 8,
  'End Year': 2021,
  'End Month': 8,
  'Exp (in months)': 12,
  'Total Exp': 13,
  'Location': 'No Location',
  'Description': 'No Description'},
 {'Title': 'Brand Ambassador',
  'Company Name': 'GradsKey',
  'Start Year': 2020,
  'Start Month': 9,
  'End Year': 2021,
  'End Month': 8,
  'Exp (in months)': 12

In [57]:
pkk['Skills'][0]

['MySQL',
 'Python (Programming Language)',
 'HTML',
 'Machine Learning',
 'Object-Oriented Programming (OOP)',
 'Data Structures',
 'SQL',
 'C (Programming Language)',
 'Microsoft PowerPoint',
 'Natural Language Processing (NLP)',
 'JavaScript',
 'Flutter',
 'Time Management',
 'Algorithms',
 'C++',
 'Microsoft Excel',
 'Microsoft Word',
 'Interpersonal Skills']

In [58]:
pkk['Certificates'][0]

[{'Name': 'Reading Financial Statements',
  'Authority': 'Corporate Finance Institute® (CFI)'},
 {'Name': 'Fundamentals of digital marketing',
  'Authority': 'Google Digital Unlocked'},
 {'Name': 'Excel Skills for Business Specialization',
  'Authority': 'Macquarie University'},
 {'Name': 'Data Structures and Algorithms', 'Authority': 'GradsKey'},
 {'Name': 'FUNDAMENTALS OF DEEP LEARNING', 'Authority': 'NVIDIA'},
 {'Name': 'FUNDAMENTALS OF ACCELERATED DATA SCIENCE WITH RAPIDS',
  'Authority': 'NVIDIA'},
 {'Name': 'Financial Markets (with Honors)', 'Authority': 'Yale University'},
 {'Name': 'Data Science Professional Certificate', 'Authority': 'IBM'}]

In [59]:
pkk['Projects'][0]

[{'Title': 'Low code No code applications',
  'Description': 'Built 3 applications on retool. Learnt about creating dashboards, integrating APIs (Twilio and Slack) and handling JSON objects.'},
 {'Title': 'Healthcare Appointment Scheduling Application',
  'Description': 'Handled Project Flow management and Implementation of Trie Data Structure for searching doctors.'},
 {'Title': 'Car Rental Application in C++',
  'Description': 'Learnt about OOP in C++ and file handling.'}]

In [60]:
pkk['Vol Exp'][0]

[]

In [61]:
pkk['Honors'][0]

[]